In [1]:
# set a variable that targets the image folder in it
SV_DATA = 'E:\compcars\sv_data_20_classes\image'

In [2]:
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim

# Define data transformations and directories
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    # we will be doing this analysis in grayscale
    transforms.Grayscale()
])

# Create a dataset using ImageFolder
dataset = torchvision.datasets.ImageFolder(root=SV_DATA, transform=transform)

# Create data loaders for training and validation
batch_size = 32
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
# split this up again for val and test dataset
test_size = int(0.5 * len(val_dataset))
val_size = len(val_dataset) - test_size
val_dataset, test_dataset = torch.utils.data.random_split(val_dataset, [val_size, test_size])

# initializaing the dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [3]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    # note that this neural network needs to be able to accept the 20 classes that
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        # defining the feature extraction layer of the network, these are convolution layers, they are paired with activation
        # layers, which adds non-linearity into the network, allowing it to learn better
        # feel free to edit the layers and see how they affect the network
        self.features = nn.Sequential(
            # this makes it easier to make layers of the neural network
            # as an excercise, try calculating the output of each layer, for example
            # input of 32, 1, 100, 100 (batch size, channel, width, height)
            nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(7,7), stride=(1,1 )),
            # 32, 3, 94, 94
            nn.ReLU(),
            # 32, 3, 94, 94
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
            # 32, 3, 47, 47

            nn.Conv2d(in_channels=3, out_channels=12, kernel_size=(10, 10), stride=(1,1 )),
            #
            nn.ReLU(),
            #
            nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)),
            #

            nn.Conv2d(in_channels=12, out_channels=60, kernel_size=(5, 5), stride=(2, 2)),
            #
            nn.ReLU()
            #
        )

        # defining the classifier of the network, where the network makes a decision of what class the image is
        # based on the features extracted
        self.classifier = nn.Sequential(
            nn.Linear(3840, 20)
        )
    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out, 1)   # this one is for the batches, to resize it so that it wont have an issue
        out = self.classifier(out)
        return out

In [9]:
model = NeuralNetwork()
from torchinfo import summary
from tqdm import tqdm

In [10]:
# defining the parameters of the network
optimizer = optim.SGD(model.parameters(), lr=0.001) # this is the learning rate
criterion = nn.CrossEntropyLoss()                   # this is the loss function

max_epoch = 3    # how long we want it to train

# you can change the runtime type at the top right corner. i used cpu for testing this notebook
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('cuda')
else:
    device = torch.device('cpu')
    print('cpu')

cuda


In [12]:
import time
start_time = time.time()

# casting the model to whichever device
model.to(device)

# these will be to log the data we collect
epoch_accuracy_values_train = []
epoch_loss_values_train = []
epoch_accuracy_values_eval = []
epoch_loss_values_eval = []

best_epoch_value = 0    # this is to see the highest accuracy
best_epoch_epoch = 0    # this is to see when it achieves it

# this will be looping through the epochs now
for currentEpoch in range(max_epoch):
    accuracy_values = []
    loss_values = []
    print(f'training started for epoch: {currentEpoch}')

    # we will be going through the training dataset
    for idx, (data, label) in enumerate(tqdm(train_loader)):
        # this try statement is to catch any weird errors, so that if anything odd breaks, the training can still continue
        try:
            data, label = data.to(device), label.to(device)

            # getting the outputs
            outputs = model(data)

            loss = criterion(outputs, label)

            # updating the weights of the model
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # logging the varaibles
            # loss
            loss_values.append((loss.item()))

            # accuracy
            _, predicted = torch.max(outputs, 1) # in python _ means a variable that we are not going to use
            # so the system automatically throws it away for us
            accuracy = (predicted == label).float().mean()
            accuracy_values.append(accuracy)

        except Exception as e:
            print(f'error located at index: {str(idx)} at epoch {str(currentEpoch)}')

    # after each pass through the dataloader (one epoch), calculate the accuracy and loss for that epoch
    epoch_loss = np.mean(loss_values)
    epoch_loss_values_train.append(epoch_loss)
    epoch_accuracy = torch.mean(torch.stack(accuracy_values))   # due to it being tensor
    epoch_accuracy_values_train.append(epoch_accuracy.item())

    tempString = 'currently at epoch ' + str(currentEpoch) + ' train accuracy: ' + str(epoch_accuracy) + ' loss of: ' + str(epoch_loss)
    print(tempString)

    # going through the evaluation phase to see how well it performs, and whether it overfitted or not
    with torch.no_grad(): # this one turns off the process that allows the network to train. This is because we only want to evaluate it, not train it right now
        for idx, (data, label) in enumerate(tqdm(val_loader)):
            # this try statement is to catch any weird errors, so that if anything odd breaks, the training can still continue
            try:
                data, label = data.to(device), label.to(device)

                # getting the outputs
                outputs = model(data)

                loss = criterion(outputs, label)


                # logging the varaibles
                # loss
                loss_values.append((loss.item()))

                # accuracy
                _, predicted = torch.max(outputs, 1) # in python _ means a variable that we are not going to use
                # so the system automatically throws it away for us
                accuracy = (predicted == label).float().mean()
                accuracy_values.append(accuracy)

            except Exception as e:
                print(f'error located at index: {str(idx)} at epoch {str(currentEpoch)}')
        # after each pass through the dataloader (one epoch), calculate the accuracy and loss for that epoch
        epoch_loss = np.mean(loss_values)
        epoch_loss_values_eval.append(epoch_loss)
        epoch_accuracy = torch.mean(torch.stack(accuracy_values))   # due to it being tensor
        epoch_accuracy_values_eval.append(epoch_accuracy.item())

        tempString = 'currently at epoch ' + str(currentEpoch) + ' eval accuracy: ' + str(epoch_accuracy) + ' loss of: ' + str(epoch_loss)
        print(tempString)

end_time = time.time()
time_taken = end_time - start_time
print(f'the time taken is: {time_taken}')

training started for epoch: 0


100%|██████████| 65/65 [00:40<00:00,  1.60it/s]


currently at epoch 0 train accuracy: tensor(0.1010, device='cuda:0') loss of: 2.9709756007561317


100%|██████████| 9/9 [00:05<00:00,  1.69it/s]


currently at epoch 0 eval accuracy: tensor(0.0997, device='cuda:0') loss of: 2.9712329265233635
training started for epoch: 1


...

currently at epoch 1 eval accuracy: tensor(0.1175, device='cuda:0') loss of: 2.914575009732633
training started for epoch: 2


100%|██████████| 65/65 [00:13<00:00,  4.66it/s]


currently at epoch 2 train accuracy: tensor(0.1212, device='cuda:0') loss of: 2.868048330453726


100%|██████████| 9/9 [00:01<00:00,  5.86it/s]

currently at epoch 2 eval accuracy: tensor(0.1174, device='cuda:0') loss of: 2.879438535587208
the time taken is: 82.83855438232422
